In [25]:
!pip install spacy
!pip install sentence-transformers
!pip install vaderSentiment
!pip install rapidfuzz


In [26]:
import pandas as pd 
import numpy as np 


In [28]:
training_data = pd.read_csv('data/liar-plus/train2.tsv', sep='\t', header=None)
training_data.columns =['index','id', 'label', 'statement', 'subject', 'speaker', 'speaker_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context','justification']
training_data.drop(columns=['index'], inplace=True)
training_data.head()
training_data['party_affiliation'].unique()
party_map = {
    'republican': 'right-leaning',
    'democrat': 'left-leaning',
    'libertarian': 'right-leaning',
    'tea-party-member': 'right-leaning',
    'ocean-state-tea-party-action': 'right-leaning',
    'constitution-party': 'right-leaning',
    'democratic-farmer-labor': 'left-leaning',
    'green': 'left-leaning',
    'labor-leader': 'left-leaning',
    'liberal-party-canada': 'centrist',
    'Moderate': 'centrist',
    'independent': 'centrist',
    'none': 'other',
    'organization': 'other',
    'columnist': 'other',
    'activist': 'other',
    'talk-show-host': 'other',
    'newsmaker': 'other',
    'journalist': 'other',
    'state-official': 'other',
    'business-leader': 'other',
    'education-official': 'other',
    'government-body': 'other'
}
training_data['party_category'] = training_data['party_affiliation'].map(party_map).fillna('other')





In [29]:
training_data
df = training_data
df

,id,label,statement,subject,speaker,speaker_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification,party_category
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...,right-leaning
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe...",left-leaning
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...,left-leaning
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...,other
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start...",left-leaning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10237,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report""",They compounded their error by combining full ...,other
10238,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,"Romney said that ""Obamacare means that for up...",right-leaning
10239,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,But that it leaves out important details and t...,right-leaning
10240,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,"We checked the research and, quite frankly, fi...",left-leaning


In [30]:
df['word_count'] = df['statement'].apply(lambda x: len(x.split()) if pd.notnull(x) else 0)
df

,id,label,statement,subject,speaker,speaker_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification,party_category,word_count
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...,right-leaning,11
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe...",left-leaning,24
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...,left-leaning,19
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...,other,12
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start...",left-leaning,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10237,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report""",They compounded their error by combining full ...,other,17
10238,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,"Romney said that ""Obamacare means that for up...",right-leaning,14
10239,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,But that it leaves out important details and t...,right-leaning,28
10240,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,"We checked the research and, quite frankly, fi...",left-leaning,11


In [31]:
df['word_count'].describe()

count    10242.000000
mean        18.027338
std          9.928149
min          0.000000
25%         12.000000
50%         17.000000
75%         22.000000
max        479.000000
Name: word_count, dtype: float64

In [41]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment_analyzer = SentimentIntensityAnalyzer()
df['statement_sentiment'] = df['statement'].apply(lambda x: sentiment_analyzer.polarity_scores(str(x))['compound'])


In [42]:
datum = df.iloc[0]

In [43]:
import spacy
# spacy.cli.download("en_core_web_md")

nlp = spacy.load("en_core_web_md")
doc = nlp(datum['statement'])
doc.vector.shape

(300,)

In [44]:
from rapidfuzz import fuzz
conservative_bigrams = pd.read_csv('top_conservative_bigrams.csv')['bigram']
liberal_bigrams = pd.read_csv('top_liberal_bigrams.csv')['bigram']
def match_counter(statement, bigram_list, threshold):
    stat = nlp(str(statement))
    word = [word.text.lower() for word in stat]
    bigram_coll = [''.join(word[i:i+2]) for i in range(len(word)-1)]
    matches = 0
    for bigram in bigram_coll:
        for check in bigram_list:
            if fuzz.ratio(bigram, check) >= threshold:
                matches += 1
                break

    return matches
df['conservative_bigram_count'] = df['statement'].apply(
    lambda x: match_counter(x, conservative_bigrams, threshold=70)
)
df['liberal_bigram_count'] = df['statement'].apply(
    lambda x: match_counter(x, liberal_bigrams, threshold=70)
)


array([ 0,  2,  1,  3,  5,  6,  4, 10, 23,  7,  8, 11])

In [45]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

def chunk_text(text, chunk_size=200, overlap=50):
    words = str(text).split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

model = SentenceTransformer("all-MiniLM-L6-v2")  # small but high quality

def embed_statement(text):
    chunks = chunk_text(text)
    embeddings = model.encode(chunks)
    return np.mean(embeddings, axis=0)  # average chunk embeddings

df["embedding"] = df["statement"].apply(embed_statement)
# X = np.stack(df["embedding"])
# y = df["truth_value"]

# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
df

/Users/ryanxavier/Downloads/ai4good/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,label,statement,subject,speaker,speaker_job_title,state_info,party_affiliation,barely_true_counts,false_counts,...,mostly_true_counts,pants_on_fire_counts,context,justification,party_category,word_count,statement_sentiment,conservative_bigram_count,liberal_bigram_count,embedding
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,...,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...,right-leaning,11,0.2500,0,0,"[-0.023657007, -0.086200885, -0.044483673, 0.0..."
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,...,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe...",left-leaning,24,0.3612,1,0,"[0.01405132, 0.031802077, 0.05521153, 0.047837..."
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,...,163.0,9.0,Denver,Obama said he would have voted against the ame...,left-leaning,19,0.3182,0,0,"[-0.016568873, 0.012516667, -0.035006717, 0.02..."
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,...,5.0,44.0,a news release,The release may have a point that Mikulskis co...,other,12,0.7579,1,2,"[-0.074178554, 0.063911326, 0.03357102, 0.0084..."
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,...,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start...",left-leaning,10,0.0000,0,2,"[0.009058817, -0.0092393095, 0.05648805, 0.028..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10237,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,...,1.0,0.0,"interview on ""The Colbert Report""",They compounded their error by combining full ...,other,17,-0.7506,0,1,"[0.004385522, -0.032097675, 0.047505867, 0.048..."
10238,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,...,1.0,0.0,an interview,"Romney said that ""Obamacare means that for up...",right-leaning,14,0.4019,1,2,"[-0.023788735, 0.03023505, -0.008846388, 0.037..."
10239,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,...,3.0,3.0,a Republican presidential debate,But that it leaves out important details and t...,right-leaning,28,0.5859,1,0,"[-0.0015655393, 0.05281168, -0.041569903, 0.03..."
10240,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,...,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,"We checked the research and, quite frankly, fi...",left-leaning,11,0.0000,0,0,"[-0.079076126, 0.09082639, -0.0039216983, 0.03..."


In [46]:
!pip install xgboost

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.5 MB/s  0:00:00m eta 0:00:01


In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
drop_cols = ['id', 'justification', 'context', 'embedding','statement', 'subject', 'speaker', 'speaker_job_title', 'party_category','party_affiliation']
X = df.drop(columns=drop_cols)
X['state_info'] = X['state_info'].astype('category')
X['label'] = X['label'].astype('category')
embeddings = np.vstack(df['embedding'].values)
embedding_cols = [f'embedding_{i}' for i in range(embeddings.shape[1])]
embedding_df = pd.DataFrame(embeddings, columns=embedding_cols, index=df.index)
X = pd.concat([X, embedding_df], axis=1)
le = LabelEncoder()
y = le.fit_transform(df['party_category'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=0)
# xgb_train = xgb.DMatrix(X_train, y_train, enable_categorical=True)
# xgb_test = xgb.DMatrix(X_test, y_test, enable_categorical=True)
model = XGBClassifier(
    objective='multi:softmax',
    num_class=len(le.classes_),
    max_depth=3,
    learning_rate=0.1,
    enable_categorical=True
)

scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Cross-validated accuracy:", scores.mean())

Cross-validated accuracy: 0.768796039989019
